# **Visualizing Intermediate Activations**
Here, we visualize the intermediate activations of our Keras model.

Import necessary packages

In [ ]:
from keras.models import load_model, Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

Add your image path in the cell below

In [ ]:
img_path = 'YOUR IMAGE PATH'

Preprocess input

In [ ]:
x = plt.imread(img_path)
plt.imshow(x)
x = x/255
x = np.expand_dims(x,0)
x.shape

Load your Keras model below

In [ ]:
# Add model path below
model = load_model('YOUR MODEL PATH')

Get model predictions

In [ ]:
model.predict(x)

In [ ]:
# View all the layers of the model
model.layers

Select layer outputs and create Activation Model

In [ ]:
# Select the number of layers to take into consideration
numlayers = 79 # We consider Xception model in this case

# Extracts the outputs of the top 12 conv layers
layer_outputs = [layer.output for layer in model.layers[:79]][1:]

# Creates a model that will return these outputs, given the model input
activation_model = Model(inputs=model.input, outputs=layer_outputs)

View Activation model summary

In [ ]:
activation_model.summary()

Visualize Activation of a filter of the first layer

In [ ]:
# Get outputs of all layers of activation model for given input
activations = activation_model.predict(x)

# Get activations of first layer
first_layer_activation = activations[0]
print("Shape of first layer activation: ", first_layer_activation.shape)

# Visualize the activation of a given filter of a first layer
filter_num = 4 # Choose your filter number
plt.matshow(first_layer_activation[0, :, :, filter_num], cmap='viridis')

Visualize Activation of a filter of any intermediate layer

In [ ]:
# Get outputs of all layers of activation model for given input
activations = activation_model.predict(x)

# Get activations of intermediate layer
intermediate_layer_num = 34
int_layer_activation = activations[34]
print("Shape of first layer activation: ", first_layer_activation.shape)

# Visualize the activation of a given filter of a given layer
filter_num = 4 # Choose your filter number
plt.matshow(int_layer_activation[0, :, :, filter_num], cmap='viridis')

Create a dictionary corresponding to the layer names and their activations

In [ ]:
layer_acts = dict(zip(layer_names, ['No activation']+ activations))

In [ ]:
layer_acts.keys() # List of layer names

Visualize all the filter activations of a given layer

In [ ]:
images_per_row = 16
# Enter your layer name below
layer_name ='SELECT YOUR LAYER' # Select any layer apart from the first one

n_features = layer_acts[layer_name].shape[-1] # Number of features in the feature map
size = layer_acts[layer_name].shape[1] #The feature map has shape (1, size, size, n_features).
n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
display_grid = np.zeros((size * n_cols, images_per_row * size))

for col in range(n_cols): # Tiles each filter into a big horizontal grid
    for row in range(images_per_row):
        channel_image = layer_acts[layer_name][0, :, :, col * images_per_row + row]
        channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
        channel_image /= channel_image.std()
        channel_image *= 64
        channel_image += 128
        channel_image = np.clip(channel_image, 0, 255).astype('uint8')
        display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image # Displays the grid

scale = 1. / size
plt.figure(figsize=(scale * display_grid.shape[1],
                    scale * display_grid.shape[0]), facecolor='white')
plt.axis('off')
plt.title(layer_name)
plt.grid(False)
plt.imshow(display_grid, aspect='auto', cmap='viridis')